In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import requests
import time
import json
import os
import random

# ============================================
# A组功能：API获取函数
# ============================================

def get_up_fans(mid):
    """
    A组：获取UP主粉丝数
    使用B站API：https://api.bilibili.com/x/relation/stat?vmid={mid}
    """
    url = f"https://api.bilibili.com/x/relation/stat?vmid={mid}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer': 'https://www.bilibili.com/'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            data = response.json()
            return data.get("data", {}).get("follower", "N/A")
    except Exception as e:
        print(f"获取粉丝数API失败: {e}")
    
    return "N/A"

def get_video_comments(aid):
    """
    A组：获取视频评论数
    """
    # 使用web-interface接口（与获取弹幕数的接口相同）
    url = f"https://api.bilibili.com/x/web-interface/view?aid={aid}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer': 'https://www.bilibili.com/'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            data = response.json()
            # 从这个接口获取评论数
            return data.get("data", {}).get("stat", {}).get("reply", "N/A")
    except Exception as e:
        print(f"获取评论数API失败: {e}")
    
    return "N/A"

def get_video_danmaku(aid):
    """
    A组：获取视频弹幕数
    使用B站API：https://api.bilibili.com/x/web-interface/view?aid={aid}
    """
    url = f"https://api.bilibili.com/x/web-interface/view?aid={aid}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer': 'https://www.bilibili.com/'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            data = response.json()
            return data.get("data", {}).get("stat", {}).get("danmaku", "N/A")
    except Exception as e:
        print(f"获取弹幕数API失败: {e}")
    
    return "N/A"

# ============================================
# B组：HTML/JSON解析函数
# ============================================

def analyurl_mainhtml(url):
    """
    B组：使用Selenium获取并解析排行榜页面
    处理动态加载的页面内容
    """
    # 配置Chrome选项
    options = Options()
    options.add_argument('--headless')  # 无界面模式
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # 初始化WebDriver
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    # 访问目标页面
    driver.get(url)

    # 等待页面加载完成
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.rank-item')))

    # 获取网页的HTML内容
    html = driver.page_source

    # 关闭WebDriver
    driver.quit()

    # 解析HTML
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def download_cover(aid, url, save_dir="covers"):
    """
    B组：下载封面图片到本地
    """
    os.makedirs(save_dir, exist_ok=True)
    try:
        # 设置请求头
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
            'Referer': 'https://www.bilibili.com/',
        }
        response = requests.get(url, headers=headers, timeout=8)
        response.raise_for_status()
        
        # 生成保存路径
        save_path = f"{save_dir}/{aid}.jpg"
        
        with open(save_path, "wb") as f:
            f.write(response.content)
        
        print(f"封面下载成功: {save_path}")
        return save_path
    except Exception as e:
        print(f"封面下载失败: {e}")
        return None

def extract_tags_from_html(soup):
    """
    B组辅助函数：从HTML中提取标签
    """
    tags = []
    
    # 查找所有class为"tag-link"的a标签
    tag_links = soup.find_all('a', {'class': 'tag-link'})
    for tag_link in tag_links:
        tag_text = tag_link.text.strip()
        if tag_text and tag_text not in tags:
            tags.append(tag_text)
    
    return tags if tags else "N/A"

def parse_video_page(link, index):
    """
    B组主功能：解析单个视频页面
    包含A组API调用的整合
    """
    print(f"正在处理第 {index} 个视频: {link}")
    
    # 确保链接格式正确
    if link.startswith('//'):
        url = 'https:' + link
    elif link.startswith('/'):
        url = 'https://www.bilibili.com' + link
    else:
        url = link
    
    # 保存视频链接
    video_link = url
    
    # 初始化所有变量
    html = None
    blogger_name = "N/A"
    title = "N/A"
    tags = "N/A"
    categories = "N/A"
    play_count = "N/A"
    like_count = "N/A"
    coin_count = "N/A"
    fav_count = "N/A"
    share_count = "N/A"
    publish_time = "N/A"
    video_description = "N/A"
    duration = "N/A"  # 视频时长
    cover_url = "N/A"  # 封面URL
    cover_path = "N/A"  # 封面本地路径
    aid = "N/A"  # 视频aid，用于API调用和封面下载
    mid = "N/A"  # UP主mid，用于API调用
    fans_count = "N/A"  # 粉丝数
    comments_count = "N/A"  # 评论数
    danmaku_count = "N/A"  # 弹幕数
    
    # 设置请求头
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.43',
        'Referer': 'https://www.bilibili.com/',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Cache-Control': 'no-cache',
        'Pragma': 'no-cache'
    }
    
    # ============================================
    # 第一步：获取页面HTML内容
    # ============================================
    try:
        session = requests.Session()
        session.headers.update(headers)
        response = session.get(url, timeout=15)
        
        if response.status_code == 200:
            html = response.text
            print(f"成功获取页面内容，长度: {len(html)}")
        else:
            print(f'无法获取链接: {url}, 状态码: {response.status_code}')
            # 即使请求失败，也要写入默认值到CSV
            write_to_csv(index, video_link, blogger_name, title, tags, categories, play_count, 
                        like_count, coin_count, fav_count, share_count, publish_time, 
                        video_description, duration, cover_url, cover_path,
                        fans_count, comments_count, danmaku_count)
            return
    except Exception as e:
        print(f'获取链接时出错: {url}, 错误: {e}')
        write_to_csv(index, video_link, blogger_name, title, tags, categories, play_count, 
                    like_count, coin_count, fav_count, share_count, publish_time, 
                    video_description, duration, cover_url, cover_path,
                    fans_count, comments_count, danmaku_count)
        return

    # 检查html是否成功获取
    if html is None:
        print('未能获取到页面内容:', url)
        write_to_csv(index, video_link, blogger_name, title, tags, categories, play_count, 
                    like_count, coin_count, fav_count, share_count, publish_time, 
                    video_description, duration, cover_url, cover_path,
                    fans_count, comments_count, danmaku_count)
        return

    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # ============================================
    # 第二步：从JavaScript中提取数据（首选方法）
    # ============================================
    script_tags = soup.find_all('script')
    video_data = None
    
    for script in script_tags:
        if script.string and 'window.__INITIAL_STATE__' in script.string:
            # 提取JavaScript中的数据
            script_text = script.string
            start = script_text.find('window.__INITIAL_STATE__=') + len('window.__INITIAL_STATE__=')
            end = script_text.find(';(function()') if ';(function()' in script_text else script_text.rfind('}') + 1
            
            if start != -1 and end != 0:
                try:
                    json_str = script_text[start:end]
                    json_data = json.loads(json_str)
                    video_data = json_data.get('videoData', {})
                    print("成功从JavaScript中提取视频数据")
                    break
                except Exception as e:
                    print(f"解析JSON数据失败: {e}")
                    continue
    
    # ============================================
    # 第三步：处理提取到的数据
    # ============================================
    if video_data:
        print("使用JavaScript数据")
        
        # 提取基础信息
        aid = video_data.get('aid', 'N/A')
        title = video_data.get('title', 'N/A')
        
        # 提取UP主信息
        owner = video_data.get('owner', {})
        blogger_name = owner.get('name', 'N/A')
        mid = owner.get('mid', 'N/A')
        
        # 提取统计数据
        stat = video_data.get('stat', {})
        like_count = stat.get('like', 'N/A')
        coin_count = stat.get('coin', 'N/A')
        fav_count = stat.get('favorite', 'N/A')
        share_count = stat.get('share', 'N/A')
        play_count = stat.get('view', 'N/A')
        
        # 提取发布时间
        publish_time = video_data.get('pubdate', 'N/A')
        if publish_time != 'N/A':
            try:
                publish_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(publish_time))
            except:
                pass
        
        # 提取视频简介
        video_description = video_data.get('desc', 'N/A')
        
        # 提取视频时长
        duration = video_data.get('duration', 'N/A')
        if duration != 'N/A':
            try:
                # 将秒转换为分钟:秒格式
                minutes = duration // 60
                seconds = duration % 60
                duration = f"{minutes}:{seconds:02d}"
            except:
                pass
        
        # 提取封面URL
        cover_url = video_data.get('pic', 'N/A')
        
        # 下载封面图片
        if aid != 'N/A' and cover_url != 'N/A':
            cover_path = download_cover(aid, cover_url)
            if cover_path is None:
                cover_path = "下载失败"
        else:
            cover_path = "N/A"
        
        # 提取标签
        tags_data = video_data.get('tags', [])
        if isinstance(tags_data, list):
            tags = []
            for tag in tags_data:
                if isinstance(tag, dict):
                    tag_name = tag.get('tag_name', '')
                    if tag_name:
                        tags.append(tag_name)
                elif isinstance(tag, str):
                    tags.append(tag)
            if not tags:
                tags = extract_tags_from_html(soup)
        else:
            tags = extract_tags_from_html(soup)
        
        # 提取类别
        tname = video_data.get('tname', 'N/A')
        categories = [tname] if tname != 'N/A' else []
    
    else:
        # ============================================
        # 第四步：如果无法从JS获取数据，则从HTML中提取
        # ============================================
        print("使用HTML解析")
        
        # 提取视频标题
        title_element = soup.find('h1', {'class': 'video-title'})
        if not title_element:
            title_element = soup.find('h1', {'class': re.compile('title')})
        title = title_element.text.strip() if title_element else "N/A"
        
        # 提取博主名称
        name_element = soup.find('a', {'class': 'username'})
        if not name_element:
            name_element = soup.find('a', {'class': re.compile('name')})
            if not name_element:
                name_element = soup.find('div', {'class': 'up-name'})
        blogger_name = name_element.text.strip() if name_element else "N/A"
        
        # 提取播放量
        view_element = soup.find('span', {'class': 'view'})
        if not view_element:
            view_element = soup.find('span', {'class': re.compile('view')})
            if not view_element:
                view_element = soup.find('div', {'class': 'video-data'})
        play_count = view_element.text.strip() if view_element else "N/A"
        
        # 提取点赞数、投币数、收藏数、分享数
        tool_bar = soup.find('div', {'class': 'ops'})
        if not tool_bar:
            tool_bar = soup.find('div', {'class': re.compile('toolbar|action')})
        
        if tool_bar:
            like_element = tool_bar.find('span', {'class': 'like'})
            if not like_element:
                like_element = tool_bar.find('span', {'class': re.compile('like')})
            like_count = like_element.text.strip() if like_element else "N/A"
            
            coin_element = tool_bar.find('span', {'class': 'coin'})
            if not coin_element:
                coin_element = tool_bar.find('span', {'class': re.compile('coin')})
            coin_count = coin_element.text.strip() if coin_element else "N/A"
            
            fav_element = tool_bar.find('span', {'class': 'collect'})
            if not fav_element:
                fav_element = tool_bar.find('span', {'class': re.compile('collect|fav')})
            fav_count = fav_element.text.strip() if fav_element else "N/A"
            
            share_element = tool_bar.find('span', {'class': 'share'})
            if not share_element:
                share_element = tool_bar.find('span', {'class': re.compile('share')})
            share_count = share_element.text.strip() if share_element else "N/A"
        
        # 提取发布时间
        time_element = soup.find('div', {'class': 'video-info'})
        if not time_element:
            time_element = soup.find('div', {'class': re.compile('video-info|publish')})
            if not time_element:
                time_element = soup.find('span', {'class': 'pudate-text'})
        
        if time_element:
            time_text = time_element.text
            time_match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', time_text)
            publish_time = time_match.group(0) if time_match else "N/A"
        else:
            publish_time = "N/A"
        
        # 提取视频简介
        desc_element = soup.find('div', {'class': 'desc-info'})
        if not desc_element:
            desc_element = soup.find('div', {'class': re.compile('desc|info|introduction')})
        video_description = desc_element.text.strip() if desc_element else "N/A"
        
        # 提取标签
        tags = extract_tags_from_html(soup)

        # 提取类别
        category_elements = soup.find_all('a', href=re.compile(r'//www\.bilibili\.com/v/'))
        categories = []
        for category_element in category_elements:
            category_text = category_element.text.strip()
            if category_text and category_text not in categories:
                categories.append(category_text)
        
        # 尝试从HTML中提取时长和封面
        duration_element = soup.find('span', {'class': 'duration'})
        if not duration_element:
            duration_element = soup.find('span', {'class': re.compile('time|duration')})
        duration = duration_element.text.strip() if duration_element else "N/A"
        
        # 从HTML中提取封面URL
        meta_image = soup.find('meta', {'property': 'og:image'})
        if meta_image:
            cover_url = meta_image.get('content', 'N/A')
        
        # 从URL中提取aid
        aid_match = re.search(r'av(\d+)', url)
        if aid_match:
            aid = aid_match.group(1)
        
        # 下载封面
        if aid != 'N/A' and cover_url != 'N/A':
            cover_path = download_cover(aid, cover_url)
            if cover_path is None:
                cover_path = "下载失败"
        else:
            cover_path = "N/A"
    
    # ============================================
    # 调用API获取扩展数据
    # ============================================
    if aid != 'N/A':
        print("调用API获取扩展数据...")
        # 获取弹幕数
        danmaku_count = get_video_danmaku(aid)
        
        # 获取评论数
        comments_count = get_video_comments(aid)
        
        # 添加随机延迟，避免请求过快
        time.sleep(random.uniform(0.5, 1.5))
    
    if mid != 'N/A':
        # 获取粉丝数
        fans_count = get_up_fans(mid)
        
        # 添加随机延迟
        time.sleep(random.uniform(0.5, 1.5))
    
    # ============================================
    # 第六步：打印并保存数据
    # ============================================
    print(f"博主名称: {blogger_name}")
    print(f"视频标题: {title}")
    print(f"标签: {tags}")
    print(f"类别: {categories}")
    print(f"播放量: {play_count}")
    print(f"点赞数: {like_count}")
    print(f"投币数: {coin_count}")
    print(f"收藏数: {fav_count}")
    print(f"分享数: {share_count}")
    print(f"发布时间: {publish_time}")
    print(f"视频时长: {duration}")
    print(f"封面URL: {cover_url}")
    print(f"封面路径: {cover_path}")
    print(f"视频链接: {video_link}")
    print(f"UP主粉丝数: {fans_count}")
    print(f"视频评论数: {comments_count}")
    print(f"视频弹幕数: {danmaku_count}")
    print(f"视频简介: {video_description[:50] if video_description != 'N/A' else 'N/A'}...")

    # 写入CSV文件
    write_to_csv(index, video_link, blogger_name, title, tags, categories, play_count, 
                like_count, coin_count, fav_count, share_count, publish_time, 
                video_description, duration, cover_url, cover_path,
                fans_count, comments_count, danmaku_count)
    
    print("=" * 50)

def write_to_csv(index, video_link, blogger_name, title, tags, categories, play_count, 
                like_count, coin_count, fav_count, share_count, publish_time, 
                video_description, duration="N/A", cover_url="N/A", cover_path="N/A",
                fans_count="N/A", comments_count="N/A", danmaku_count="N/A"):
    """
    将数据写入CSV文件，包含所有字段，以及视频链接
    """
    try:
        with open('video_info_complete.csv', 'a', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)
            
            # 检查文件是否为空，如果是则写入表头
            if csvfile.tell() == 0:
                writer.writerow([
                    '序号', '视频链接', '博主名称', '视频标题', '标签', '类别', '播放量', 
                    '点赞数', '投币数', '收藏数', '分享数', '发布时间', '视频简介', 
                    '视频时长', '封面URL', '封面路径', 'UP主粉丝数', '视频评论数', '视频弹幕数'
                ])
            
            # 确保所有数据都是字符串类型
            index = str(index + 1)  # 序号从1开始
            video_link = str(video_link) if video_link is not None else "N/A"
            blogger_name = str(blogger_name) if blogger_name is not None else "N/A"
            title = str(title) if title is not None else "N/A"
            tags = str(tags) if tags is not None else "N/A"
            categories = str(categories) if categories is not None else "N/A"
            play_count = str(play_count) if play_count is not None else "N/A"
            like_count = str(like_count) if like_count is not None else "N/A"
            coin_count = str(coin_count) if coin_count is not None else "N/A"
            fav_count = str(fav_count) if fav_count is not None else "N/A"
            share_count = str(share_count) if share_count is not None else "N/A"
            publish_time = str(publish_time) if publish_time is not None else "N/A"
            video_description = str(video_description) if video_description is not None else "N/A"
            duration = str(duration) if duration is not None else "N/A"
            cover_url = str(cover_url) if cover_url is not None else "N/A"
            cover_path = str(cover_path) if cover_path is not None else "N/A"
            fans_count = str(fans_count) if fans_count is not None else "N/A"
            comments_count = str(comments_count) if comments_count is not None else "N/A"
            danmaku_count = str(danmaku_count) if danmaku_count is not None else "N/A"
            
            # 写入一行数据
            writer.writerow([
                index, video_link, blogger_name, title, tags, categories, play_count, 
                like_count, coin_count, fav_count, share_count, publish_time, 
                video_description, duration, cover_url, cover_path,
                fans_count, comments_count, danmaku_count
            ])
        
        print(f"第 {index} 个视频信息已成功写入CSV文件，包含视频链接。")
    except Exception as e:
        print(f"写入CSV文件时出错: {e}")

def extract_video_links(soup):
    """
    从排行榜页面提取视频链接
    """
    links = []
    info_divs = soup.find_all('div', class_='info')
    count = 0
    
    for info_div in info_divs:
        title_element = info_div.find('a', class_='title')
        if title_element:
            href = title_element.get('href')
            if href:
                print(f"找到链接: {href}")
                links.append(href)
                count = count + 1
                if len(links) == 100:  # 限制为100个视频
                    break
    
    print(f"链接提取完成，共找到 {count} 个链接")
    return links

def main():
    """
    主函数：整合A组和B组功能
    """
    print("=" * 60)
    print("B站视频信息爬虫 - 完整整合版")
    print("包含A组（API扩展）和B组（HTML/JSON解析）功能")
    print("=" * 60)
    
    # 检查是否已有CSV文件，如果有则备份
    if os.path.exists('video_info_complete.csv'):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        backup_name = f"video_info_complete_backup_{timestamp}.csv"
        os.rename('video_info_complete.csv', backup_name)
        print(f"已备份原有CSV文件为: {backup_name}")
    
    # 第一步：获取排行榜页面并提取视频链接
    print("\n1. 正在获取美食排行榜页面...")
    soup = analyurl_mainhtml("https://www.bilibili.com/v/popular/rank/food/")
    
    print("\n2. 正在提取视频链接...")
    urls = extract_video_links(soup)
    
    print(f"\n3. 开始爬取 {len(urls)} 个视频的详细信息...")
    
    # 第二步：逐个处理视频页面
    for index, url in enumerate(urls):
        print(f"\n处理进度: {index+1}/{len(urls)}")
        parse_video_page(url, index)
        
        # 添加延时，避免请求过快
        delay_time = random.uniform(2, 4)
        print(f"等待 {delay_time:.1f} 秒后处理下一个视频...")
        time.sleep(delay_time)
    
    print("\n" + "=" * 60)
    print("爬取完成！")
    print(f"数据已保存到: video_info_complete.csv")
    print(f"CSV文件中包含所有视频链接")
    print(f"封面图片已保存到: covers/ 文件夹")
    print("=" * 60)

# ============================================
# 程序入口
# ============================================
if __name__ == "__main__":
    # 检查是否安装所需库
    try:
        import bs4
        import selenium
        import requests
    except ImportError as e:
        print(f"缺少必要的库: {e}")
        print("请安装: pip install beautifulsoup4 selenium requests")
        exit(1)
    
    # 运行主程序
    main()

B站视频信息爬虫 - 完整整合版
包含A组（API扩展）和B组（HTML/JSON解析）功能

1. 正在获取美食排行榜页面...

2. 正在提取视频链接...
找到链接: //www.bilibili.com/video/BV1fc2eBkEMS
找到链接: //www.bilibili.com/video/BV1Fr2eBjETh
找到链接: //www.bilibili.com/video/BV1ArSsBKEZ8
找到链接: //www.bilibili.com/video/BV1T6S4ByEqT
找到链接: //www.bilibili.com/video/BV1SW2eBoEAk
找到链接: //www.bilibili.com/video/BV17V24BHEP8
找到链接: //www.bilibili.com/video/BV12o2jB3E74
找到链接: //www.bilibili.com/video/BV1Nm2YBpEeK
找到链接: //www.bilibili.com/video/BV1Qr25BCEo7
找到链接: //www.bilibili.com/video/BV16r2eBLECH
找到链接: //www.bilibili.com/video/BV17m2eBGEb7
找到链接: //www.bilibili.com/video/BV1UDUbBDEEX
找到链接: //www.bilibili.com/video/BV1Kp2aBDE2P
找到链接: //www.bilibili.com/video/BV1E42vByEQS
找到链接: //www.bilibili.com/video/BV1naSxBcEmN
找到链接: //www.bilibili.com/video/BV14e2LBMEdK
找到链接: //www.bilibili.com/video/BV1z12eBkEj1
找到链接: //www.bilibili.com/video/BV1oa25BSELU
找到链接: //www.bilibili.com/video/BV12t2VBeER8
找到链接: //www.bilibili.com/video/BV1ae2YBwEt2
找到链接: //www.bilibili.com/video/BV1nJ2v